# Импорт

In [90]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Получение ссылок с каждой страницы и информации с них

In [6]:
url = 'https://czn.mos.ru/vacancy/?WithoutAdditionalLimits=False&ActivityScopeNoStandart=True&SearchType=1&Profession=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82&Region=77&HideWithEmptySalary=False&ShowOnlyWithEmployerInfo=False&ShowOnlyWithHousing=False&ShowChukotkaResidentsVacancies=False&ShowPrimorskAreaResident1Vacancies=False&ShowPrimorskAreaResident2Vacancies=False&ShowPrimorskAreaResident3Vacancies=False&Sort=1&PageSize=0&SpecialCategories=False&IsDevelopmentProgram=False&Grid-sort=&Grid-page=1&Grid-pageSize=50&Grid-group=&Grid-filter='
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

#получаем ссылку каждой карточки с первой страницы
cards = soup.select('td:nth-child(4) > a')
urls_card = []
for link in cards:
    if link != None:
        hrefval = link.get('href')
        urls_card.append(hrefval)
        
res = []         
#меняем ссылки для получения информации и исполльзуем функцию `info_card`        
for u in urls_card:
    newUrl = 'https://czn.mos.ru/' + u
    info = info_card(newUrl)
    res.append(info)

#ищем ссылки для открывания следующих страниц
pages = soup.find('ul', class_='k-pager-numbers k-reset')
urls = []
links = pages.find_all('a', class_='k-link')

#получем ссылки каждой страницы
for link in links:
    pageNum = int(link.text) if link.text.isdigit() else None
    if pageNum != None:
        hrefval = link.get('href')
        urls.append(hrefval)
        
       
 #меняем страницы 
for slug in urls:
    newUrl = url.replace('-page=1', slug)
    response = requests.get(newUrl)
    soup = BeautifulSoup(response.text, 'lxml')
    cards = soup.select('td:nth-child(4) > a')
    #получаем информацию с карточек последующих страниц
    urls_card = []
    for link in cards:
        if link != None:
            hrefval = link.get('href')
            urls_card.append(hrefval)
    #меняем ссылки для получения информации и исполльзуем функцию `info_card`         
    for u in urls_card:
        newUrl = 'https://czn.mos.ru/' + u
        info = info_card(newUrl)
        res.append(info)
        

# Функция для сбора данных с карточки

In [5]:
def info_card(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    name_comp = soup.find('h2', class_='page-heading-title-short').text
    contact = soup.find_all('div', class_='col-sm-9') 
    res = []
    res.append(name_comp)
    for c in contact:
        res.append(c.text)
    return res

# Функция, которая по шаблонам находит нужную информацию и сохраняет найденное в датафрейм

In [86]:
def parser(filename):
    result = []
        
    name = re.search(r'(?<=\')(?P<name>.*?)(?=\')', str(filename))
    contact = re.search(r'(?<=\'\,\s\')(?P<contact>.*?)(?=\')', str(filename))
    email = re.search(r'(([a-z0-9_\.-]+)@([a-z0-9_\.-]+)\.([a-z\.]{2,6}))', str(filename))
    address = re.search(r'((\d{6}\,|(г\.))(?P<name>.*?)(?=\'))', str(filename)) 
    phone = re.search(r'((\()(?P<name>.*?)(?=\'))', str(filename))
    if email is None:
        email = ' '
        result.append([
                name.group(1),
                contact.group(1),
                email,
                address.group(1),
                phone.group(1) 
                ]) 
    elif phone is None:
        phone = ' '
        result.append([
                name.group(1),
                contact.group(1),
                email.group(1),
                address.group(1),
                phone 
                ]) 
    
    else:
            result.append([
                name.group(1),
                contact.group(1),
                email.group(1),
                address.group(1),
                phone.group(1) 
                ])  
    
 
    #сохранение полученных данных в датафрейм
    df = pd.DataFrame(result)
    df.columns = ["name", "contact", "email", 'address', "phone"]
            
    return df

# Соединение всех датафреймов в один

In [87]:
DF= []
for i in res:
    try:
        f = parser(i)
        DF.append(f)
    except Exception:
        continue
        
df = pd.concat(DF, ignore_index=True)

In [88]:
df

,name,contact,email,address,phone
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЦЕНТР ПРОГРАММ ЛОЯЛЬНОСТИ""",Затонская Екатерина Владимировна,dvgamulin@spasibosb.ru,"121170, г. Москва, ул Поклонная, д. 3, этаж 3/...",(495 ) 2122125
1,"Общество с ограниченной ответственностью ""Лента""",Егорова Елена Борисовна,,"121108, г. Москва, ул Ивана Франко, д. 8",(495 ) 6600950 доб. 702
2,"Общество с ограниченной ответственностью ""Севе...",Старшинова Анна Олеговна,fofi.severgrupp.tt@severstal-ssc.ru,"127018, г. Москва, ул Сущёвский Вал, д. 18, эт...",
3,"Акционерное общество ""Машиностроительное произ...",Тягунова Елена Сергеевна,alexpn@mporum.ru,"127015, г. Москва, ул Расковой, д. 34",(495 ) 5021922 доб. 3146
4,Акционерное общество Московский завод плавлены...,Калашникова Марина Ильинична,info@karatsc.ru,"127254, г. Москва, ул Руставели, д. 14",(495 ) 9334553 доб. 531
...,...,...,...,...,...
176,Федеральное государственное казенное образоват...,Царькова Любовь Викторовна,amvd@mail.ru,"125171, г. Москва, ул Зои и Александра Космоде...",(499 ) 7458212
177,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,Есакова Марианна Николаевна,priemnaya.givc@givc.mkrf.ru,"105066, г. Москва, ул Басманная Стар., д. 19",(495 ) 7486070 доб. 2812
178,"Общество с ограниченной ответственностью ""Цент...",Горьков Кирилл Юрьевич,info@technospark.ru,"108841, г Троицк, ул Промышленная, д. 2, корп....",(ОБЯЗАТЕЛЬНО СОЗВАНИВАТЬСЯ ДЛЯ НАЗНАЧЕНИЯ СОБЕ...
179,"Публичное акционерное общество ""Аэрофлот-росси...",Журавлева Татьяна Владимировна,rabota@aeroflot.ru,"119019, г. Москва, ул Арбат, д. 1, Обязательно...",(499 ) 5006704


In [89]:
df.to_csv('my_work.csv')